In [1]:
import pickle
import sys
import re
from unidecode import unidecode
import numpy as np
import pandas as pd
#import psycopg2
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, date
from datetime import timedelta


# Have to deal with Mexico city games having no home team

In [3]:
current_year = 2020

In [4]:
full_df = pickle.load(open(f'full_df_{current_year}.p','rb'))
dups = full_df[full_df.game_ids.duplicated(keep='first')].sort_values(by='game_ids')
dups['note'] = 'mexico_city_game'
full_df = full_df[~full_df.game_ids.duplicated(keep=False)]
full_df = full_df.append(dups)

/Users/Kevin/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [5]:
full_df

,date,game_ids,home_team,home_team_score,note,visit_team,visit_team_score
0,2020-10-23,401160630,phi,107,NaN,bos,93
1,2020-10-25,401160659,bos,112,NaN,tor,106
2,2020-10-26,401160671,ny,95,NaN,bos,118
3,2020-10-30,401160700,bos,116,NaN,mil,105
4,2020-11-01,401160714,bos,104,NaN,ny,102
...,...,...,...,...,...,...,...
44,2021-01-22,401161305,no,117,NaN,sa,121
69,2021-08-09,401224762,no,113,NaN,sa,122
23,2020-12-12,401161011,dal,122,mexico_city_game,det,111
24,2020-12-14,401161022,sa,121,mexico_city_game,phx,119


In [28]:
year=int(input("year? "))

#Removed path
games1= pickle.load(open('full_df.p','rb'))
games1.set_index('game_ids',inplace=True)

BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'
print(games1.index[0])
request = requests.get(BASE_URL.format(games1.index[0]))

table = bs(request.text,'lxml').find('table', class_='mod-data')
heads = table.find_all("thead")

headers = heads[0].find_all('tr')[0].find_all('th')[1:]
headers = [th.text for th in headers]
columns = ['id', 'team', 'player'] + headers


players = pd.DataFrame(columns=columns)

year? 2019
401160630


# OLD

In [62]:
games1.head()

,date,home_team,visit_team,home_team_score,visit_team_score
game_ids,,,,,
401160630,2019-10-23,phi,bos,107,93
401160659,2019-10-25,bos,tor,112,106
401160671,2019-10-26,ny,bos,95,118
401160700,2019-10-30,bos,mil,116,105
401160714,2019-11-01,bos,ny,104,102


In [8]:
full_df.head()

,date,game_ids,home_team,home_team_score,note,visit_team,visit_team_score
0,2020-10-23,401160630,phi,107,NaN,bos,93
1,2020-10-25,401160659,bos,112,NaN,tor,106
2,2020-10-26,401160671,ny,95,NaN,bos,118
3,2020-10-30,401160700,bos,116,NaN,mil,105
4,2020-11-01,401160714,bos,104,NaN,ny,102


In [15]:
full_df.reset_index(inplace=True,drop=True)

In [95]:
BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'
request = requests.get(BASE_URL.format(full_df.game_ids[0]))

table = bs(request.text,'lxml').find('table', class_='mod-data')
heads = table.find_all("thead")

headers = heads[0].find_all('tr')[0].find_all('th')[1:]
headers = [th.text for th in headers]
columns = ['id', 'team', 'player', 'pos'] + headers

In [96]:
table = bs(request.text,'lxml').find_all('table', class_='mod-data')
heads1 = table[0].find_all('thead')
bodies1 = table[0].find_all('tbody')
heads2 = table[1].find_all('thead')
bodies2 = table[1].find_all('tbody')

In [97]:
headers

['MIN',
 'FG',
 '3PT',
 'FT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TO',
 'PF',
 '+/-',
 'PTS']

In [109]:
'G'.isupper()

True

In [165]:
def get_players(players, team_name):
    array = np.zeros((len(players), len(headers)+2), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        
        #this is the old code
        print(cols[0].text.split(',')[0])
        print(cols[1].text)
        #print(cols[0].a['href'])
        #this is the replacement
        try:
            array[i, 0] = cols[0].a['href'].split('/')[-1]
            text_pos = cols[0].text.split(',')[0]
            
            if text_pos[-2].isupper():
                pos = text_pos[-2:]
            else:
                pos = text_pos[-1]

            array[i,1] = pos
            
            for j in range(1, len(headers) + 2):
                if not cols[1].text.startswith('DNP'):
                    array[i, j+1] = cols[j].text

            
        except:
            pass

    frame = pd.DataFrame(columns=columns)
    for x in array:
        line = np.concatenate(([full_df.game_ids[0], team_name], x)).reshape(1,len(columns))
        new = pd.DataFrame(line, columns=frame.columns)
        frame = frame.append(new)
    return frame

In [166]:
#for i,player in enumerate(team_1_players[:1]):
#    print(player.find_all('td')[0].a['href'].split("/")[-1])

In [167]:
full_df.game_ids[0]

'401160630'

In [168]:
team_1_players = bodies1[0].find_all('tr') + bodies1[1].find_all('tr')

get_players(team_1_players, 'bos')

L. KornetL. KornetPF
34
N. VonlehN. VonlehF
25
K. Knox IIK. Knox IISF
31
T. Hardaway Jr.T. Hardaway Jr.SG
25
E. MudiayE. MudiayG
31
M. HezonjaM. HezonjaF
20
M. RobinsonM. RobinsonC
10
F. NtilikinaF. NtilikinaPG
17
D. DotsonD. DotsonSG
25
A. TrierA. TrierG
23
L. ThomasL. ThomasF
DNP-COACH'S DECISION
T. BurkeT. BurkePG
DNP-COACH'S DECISION
C. LeeC. LeeG
DNP-COACH'S DECISION
TEAM

 



,id,team,player,pos,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,+/-,PTS
0,401160630,bos,luke-kornet,PF,34,5-11,4-6,2-2,1,2,3,2,5,1,2,1,-2,16
0,401160630,bos,noah-vonleh,F,25,6-11,0-1,1-2,1,9,10,1,0,0,0,0,+6,13
0,401160630,bos,kevin-knox-ii,SF,31,2-11,0-6,1-2,1,3,4,1,0,0,0,2,-7,5
0,401160630,bos,tim-hardaway-jr,SG,25,2-7,2-4,2-2,0,2,2,1,0,0,3,2,-7,8
0,401160630,bos,emmanuel-mudiay,G,31,11-18,1-3,2-2,1,6,7,2,0,0,5,2,-12,25
0,401160630,bos,mario-hezonja,F,20,0-4,0-1,0-0,0,6,6,1,0,0,1,3,-6,0
0,401160630,bos,mitchell-robinson,C,10,2-2,0-0,0-0,2,0,2,0,0,1,0,1,+1,4
0,401160630,bos,frank-ntilikina,PG,17,1-7,1-2,0-0,0,3,3,2,0,1,1,1,+11,3
0,401160630,bos,damyean-dotson,SG,25,5-8,3-4,0-0,0,3,3,2,0,0,0,2,+7,13
0,401160630,bos,allonzo-trier,G,23,5-8,1-2,2-2,0,2,2,3,1,0,2,2,+4,13


In [ ]:
# Old code I'm trying to preserve for a bit
def get_players(players, team_name):
    array = np.zeros((len(players), len(headers)+1), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        array[i, 0] = cols[0].text.split(',')[0]
        for j in range(1, len(headers) + 1):
            if not cols[1].text.startswith('DNP'):
                array[i, j] = cols[j].text

In [196]:
year=2017

#Removed path
games1 = pickle.load(open(f'full_df_{year}.p','rb'))
dups = games1[games1.game_ids.duplicated(keep='first')].sort_values(by='game_ids')
dups['note'] = 'mexico_city_game'
games1 = games1[~games1.game_ids.duplicated(keep=False)]
games1 = games1.append(dups)
games1.set_index('game_ids',inplace=True)


BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'
print(games1.index[0])
request = requests.get(BASE_URL.format(games1.index[0]))

table = bs(request.text,'lxml').find('table', class_='mod-data')
heads = table.find_all("thead")

headers = heads[0].find_all('tr')[0].find_all('th')[1:]
headers = [th.text for th in headers]
columns = ['id', 'team', 'player','pos'] + headers


players = pd.DataFrame(columns=columns)

def get_players(players, team_name):
    array = np.zeros((len(players), len(headers)+2), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        try:
            array[i, 0] = cols[0].a['href'].split('/')[-1]
            
            text_pos = cols[0].text.split(',')[0]
            
            if text_pos[-2].isupper():
                pos = text_pos[-2:]
            else:
                pos = text_pos[-1]

            array[i,1] = pos            
            
            for j in range(1, len(headers) + 2):
                if not cols[1].text.startswith('DNP'):
                    array[i, j+1] = cols[j].text
                    

        except:
            pass

    frame = pd.DataFrame(columns=columns)
    for x in array:
        line = np.concatenate(([index, team_name], x)).reshape(1,len(columns))
        new = pd.DataFrame(line, columns=frame.columns)
        frame = frame.append(new)
    return frame

for index, row in games1.iterrows():
    try:
        print(index)
        request = requests.get(BASE_URL.format(index))
        
        table = bs(request.text,'lxml').find_all('table', class_='mod-data')
        heads1 = table[0].find_all('thead')
        bodies1 = table[0].find_all('tbody')
        heads2 = table[1].find_all('thead')
        bodies2 = table[1].find_all('tbody')


        #team_1 = heads[0].th.text #trouble with getting team names
        team_1 = games1['visit_team'][index]

        team_1_players = bodies1[0].find_all('tr') + bodies1[1].find_all('tr')

        team_1_players = get_players(team_1_players, team_1)
        players = players.append(team_1_players)


        #team_2 = heads[3].th.text #trouble with getting team names
        team_2 = games1['home_team'][index]
        #team_2_players = bodies[3].find_all('tr') + bodies[4].find_all('tr')
        team_2_players = bodies2[0].find_all('tr') + bodies2[1].find_all('tr')
        team_2_players = get_players(team_2_players, team_2)
        players = players.append(team_2_players)
    except Exception as e:
        print(e)
        pass

players = players.set_index('id')
#players['position'] = players.player.apply(lambda x: x[-2:]  if x[-1] != 'C' else x[-1])
#players['player'] = players.player.apply(lambda x: x[:-2] if x[-1] != 'C' else x[:-1])

#Trying something else

def conversion(x):
    if type(x)!=float:
        return unidecode(x)
    else:
        return x

players=players.applymap(conversion)

#Remove some useless nulls
players1 = players[pd.notnull(players.MIN)]

players1.to_csv(f'/Users/Kevin/Desktop/fanduel/boxscores_{year}.csv')



/Users/Kevin/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


400899380
400899380
400899422
400899477
400899511
400899518
400899541
400899557
400899575
400899580
400899598
400899611
400899629
400899638
400899655
400899667
400899675
400899706
400899717
400899447
400899736
400899753
400899764
400899781
400899799
400899825
400899833
400899852
400899866
400899882
400899887
400899898
400899914
400899930
400899936
400899423
400899967
400899974
400899993
400900004
400900018
400900039
400900051
400900071
400900097
400900102
400900118
400900127
400900142
400899391
400900156
400900171
400900198
400900203
400900220
400900231
400900242
400900255
400900264
400900285
400900291
400899432
400900313
400900324
400900340
400900354
400900365
400900380
400900401
400900417
400900432
400900442
400900455
400900471
400900488
400900508
400900524
400899438
400900547
400900555
400900567
400900582
400900597
400899452
400899481
400899493
400899507
400899522
400899551
400899556
400899589
400899602
400899607
400899628
400899644
400899684
400899697
400899711
400899367
400899733


400900423
400900444
400900486
400900495
400900507
400900530
400899406
400900545
400900564
400899376
400899426
400899504
400899513
400899526
400899538
400899553
400899565
400899578
400899621
400899627
400899687
400899701
400899740
400899775
400899808
400899847
400899870
400899878
400899896
400899926
400899941
400899955
400899963
400899998
400900019
400900035
400900050
400900110
400900124
400900139
400900149
400900159
400900186
400900235
400900249
400900256
400900378
400900383
400900397
400900406
400900424
400900436
400900491
400900520
400899401
400900542
400900572
400900587
400900608
400899458
400899485
400899503
400899512
400899527
400899558
400899574
400899586
400899601
400899634
400899681
400899715
400899434
400899755
400899762
400899777
400899796
400899840
400899849
400899869
400899876
400899917
400899948
400899985
400900025
400900041
400900069
400900114
list index out of range
400900129
400900165
400900177
400900194
400900202
list index out of range
400900233
400900283
400900326
40

In [197]:
players

,team,player,pos,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,+/-,PTS
id,,,,,,,,,,,,,,,,,
400899380,bkn,trevor-booker,F,24,4-5,0-1,0-0,1,6,7,5,3,0,1,0,+5,8
400899380,bkn,rondae-hollis-jefferson,F,24,1-3,0-1,3-4,0,2,2,0,1,0,1,1,-9,5
400899380,bkn,bojan-bogdanovic,SF,23,9-17,3-8,0-0,2,3,5,0,1,0,2,2,+2,21
400899380,bkn,brook-lopez,C,21,1-7,1-2,4-4,2,3,5,0,0,0,2,1,-10,7
400899380,bkn,jeremy-lin,G,28,6-14,2-5,4-5,1,2,3,3,0,1,2,1,-3,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400900026,sa,jared-dudley,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400900026,sa,alan-williams,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400900026,sa,brandon-knight,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
for i in players1.columns:
    try:
        players1[i] = players1[i].replace('--',np.nan).replace('',np.nan).astype('float')
    except Exception as e:
        print (e,i)
        

could not convert string to float: 'bkn' team
could not convert string to float: 'trevor-booker' player
could not convert string to float: 'F' pos
could not convert string to float: 'Did not play' MIN
could not convert string to float: '4-5' FG
could not convert string to float: '0-1' 3PT
could not convert string to float: '0-0' FT


/Users/Kevin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [199]:
probs = []
for i in players1.PTS:
    try:
        float(i)
    except:
        if i not in probs:
            probs.append(i)

In [200]:
probs

[]

In [201]:
players1.columns

Index(['team', 'player', 'pos', 'MIN', 'FG', '3PT', 'FT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', '+/-', 'PTS'],
      dtype='object')

In [202]:
players1[players1.player == 'damian-lillard'].PTS.mean()

26.986666666666668

In [203]:
players1

,team,player,pos,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,+/-,PTS
id,,,,,,,,,,,,,,,,,
400899380,bkn,trevor-booker,F,24,4-5,0-1,0-0,1.0,6.0,7.0,5.0,3.0,0.0,1.0,0.0,5.0,8.0
400899380,bkn,rondae-hollis-jefferson,F,24,1-3,0-1,3-4,0.0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,-9.0,5.0
400899380,bkn,bojan-bogdanovic,SF,23,9-17,3-8,0-0,2.0,3.0,5.0,0.0,1.0,0.0,2.0,2.0,2.0,21.0
400899380,bkn,brook-lopez,C,21,1-7,1-2,4-4,2.0,3.0,5.0,0.0,0.0,0.0,2.0,1.0,-10.0,7.0
400899380,bkn,jeremy-lin,G,28,6-14,2-5,4-5,1.0,2.0,3.0,3.0,0.0,1.0,2.0,1.0,-3.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400900026,sa,pj-tucker,PF,30,2-7,1-5,2-2,0.0,5.0,5.0,2.0,4.0,0.0,1.0,3.0,6.0,7.0
400900026,sa,dragan-bender,F,21,2-4,2-3,0-0,0.0,6.0,6.0,1.0,0.0,0.0,0.0,5.0,6.0,6.0
400900026,sa,alex-len,C,21,1-2,0-1,2-2,1.0,3.0,4.0,0.0,0.0,0.0,1.0,3.0,-1.0,4.0


In [61]:
players.shape

(30869, 17)

In [59]:
players.loc['401161022'].iloc[0].team

401161022     sa
401161022    phx
Name: visit_team, dtype: object

In [60]:
players.loc['401160990'].iloc[0].team

'det'

In [50]:
for j in range(len(players)):
    print(players.iloc[j].name, players.iloc[j].player)
    [unidecode(i) for i in players.iloc[j] if type(i) != float]

401160630 J. TatumJ. Tatum
401160630 G. HaywardG. Hayward
401160630 E. KanterE. Kanter
401160630 K. WalkerK. Walker
401160630 J. BrownJ. Brown
401160630 S. OjeleyeS. Ojeleye
401160630 G. WilliamsG. Williams
401160630 D. TheisD. Theis
401160630 R. Williams IIIR. Williams III
401160630 B. WanamakerB. Wanamaker
401160630 M. SmartM. Smart
401160630 C. EdwardsC. Edwards
401160630 V. PoirierV. Poirier
401160630 TE
401160630 
401160630 A. HorfordA. Horford
401160630 T. HarrisT. Harris
401160630 J. EmbiidJ. Embiid
401160630 B. SimmonsB. Simmons
401160630 J. RichardsonJ. Richardso
401160630 M. ScottM. Scott
401160630 J. Ennis IIIJ. Ennis III
401160630 K. O'QuinnK. O'Quinn
401160630 R. NetoR. Neto
401160630 M. ThybulleM. Thybulle
401160630 S. MiltonS. Milton
401160630 F. KorkmazF. Korkmaz
401160630 T. BurkeT. Burke
401160630 TE
401160630 
401160659 P. SiakamP. Siakam
401160659 O. AnunobyO. Anunoby
401160659 M. GasolM. Gasol
401160659 K. LowryK. Lowry
401160659 F. VanVleetF. VanVleet
401160659 S.

401160895 B. WanamakerB. Wanamaker
401160895 C. EdwardsC. Edwards
401160895 J. GreenJ. Green
401160895 TE
401160895 
401160909 J. TatumJ. Tatum
401160909 D. TheisD. Theis
401160909 K. WalkerK. Walker
401160909 M. SmartM. Smart
401160909 J. BrownJ. Brown
401160909 S. OjeleyeS. Ojeleye
401160909 G. WilliamsG. Williams
401160909 E. KanterE. Kanter
401160909 R. Williams IIIR. Williams III
401160909 B. WanamakerB. Wanamaker
401160909 C. EdwardsC. Edwards
401160909 J. GreenJ. Green
401160909 V. PoirierV. Poirier
401160909 TE
401160909 
401160909 T. PrinceT. Prince
401160909 J. HarrisJ. Harris
401160909 J. AllenJ. Allen
401160909 S. DinwiddieS. Dinwiddie
401160909 G. TempleG. Temple
401160909 D. MusaD. Mus
401160909 D. JordanD. Jordan
401160909 I. ShumpertI. Shumpert
401160909 T. PinsonT. Pinson
401160909 H. EllensonH. Ellenson
401160909 N. ClaxtonN. Claxton
401160909 D. NwabaD. Nwaba
401160909 T. Luwawu-CabarrotT. Luwawu-Cabarrot
401160909 TE
401160909 
401160927 J. TatumJ. Tatum
401160927 D

401161115 K. WalkerK. Walker
401161115 J. BrownJ. Brown
401161115 G. WilliamsG. Williams
401161115 E. KanterE. Kanter
401161115 B. WanamakerB. Wanamaker
401161115 M. SmartM. Smart
401161115 C. EdwardsC. Edwards
401161115 J. GreenJ. Green
401161115 R. LangfordR. Langford
401161115 S. OjeleyeS. Ojeleye
401161115 TE
401161115 
401161139 J. TatumJ. Tatum
401161139 G. HaywardG. Hayward
401161139 D. TheisD. Theis
401161139 K. WalkerK. Walker
401161139 M. SmartM. Smart
401161139 S. OjeleyeS. Ojeleye
401161139 G. WilliamsG. Williams
401161139 E. KanterE. Kanter
401161139 B. WanamakerB. Wanamaker
401161139 J. GreenJ. Green
401161139 R. LangfordR. Langford
401161139 TE
401161139 
401161139 P. WashingtonP. Washington
401161139 M. BridgesM. Bridges
401161139 B. BiyomboB. Biyombo
401161139 T. RozierT. Rozier
401161139 D. GrahamD. Graham
401161139 M. WilliamsM. William
401161139 C. MartinC. Martin
401161139 C. ZellerC. Zeller
401161139 W. HernangomezW. Hernangomez
401161139 D. BaconD. Bacon
40116113

401161290 TE
401161290 
401161299 J. Jackson Jr.J. Jackson Jr.
401161299 J. CrowderJ. Crowder
401161299 J. ValanciunasJ. Valanciunas
401161299 J. MorantJ. Morant
401161299 D. BrooksD. Brooks
401161299 S. HillS. Hill
401161299 K. AndersonK. Anderson
401161299 B. ClarkeB. Clarke
401161299 T. JonesT. Jones
401161299 G. AllenG. Allen
401161299 D. MeltonD. Melton
401161299 M. GuduricM. Guduric
401161299 TE
401161299 
401161299 J. TatumJ. Tatum
401161299 G. HaywardG. Hayward
401161299 D. TheisD. Theis
401161299 K. WalkerK. Walker
401161299 M. SmartM. Smart
401161299 S. OjeleyeS. Ojeleye
401161299 G. WilliamsG. Williams
401161299 E. KanterE. Kanter
401161299 V. PoirierV. Poirier
401161299 B. WanamakerB. Wanamaker
401161299 C. EdwardsC. Edwards
401161299 J. GreenJ. Green
401161299 TE
401161299 
401161313 G. WilliamsG. Williams
401161313 G. HaywardG. Hayward
401161313 D. TheisD. Theis
401161313 K. WalkerK. Walker
401161313 M. SmartM. Smart
401161313 S. OjeleyeS. Ojeleye
401161313 V. PoirierV. P

401161472 J. EvansJ. Evans
401161472 TE
401161472 
401161484 J. TatumJ. Tatum
401161484 G. HaywardG. Hayward
401161484 D. TheisD. Theis
401161484 M. SmartM. Smart
401161484 J. BrownJ. Brown
401161484 S. OjeleyeS. Ojeleye
401161484 G. WilliamsG. Williams
401161484 E. KanterE. Kanter
401161484 V. PoirierV. Poirier
401161484 B. WanamakerB. Wanamaker
401161484 R. LangfordR. Langford
401161484 C. EdwardsC. Edwards
401161484 J. GreenJ. Green
401161484 TE
401161484 
401161484 A. DavisA. Davis
401161484 L. JamesL. James
401161484 J. McGeeJ. McGee
401161484 A. BradleyA. Bradley
401161484 D. GreenD. Gree
401161484 K. KuzmaK. Kuzma
401161484 D. HowardD. Howard
401161484 R. RondoR. Rondo
401161484 K. Caldwell-PopeK. Caldwell-Pope
401161484 A. CarusoA. Caruso
401161484 J. DudleyJ. Dudley
401161484 Q. CookQ. Cook
401161484 T. DanielsT. Daniels
401161484 TE
401161484 
401161504 J. TatumJ. Tatum
401161504 G. HaywardG. Hayward
401161504 D. TheisD. Theis
401161504 M. SmartM. Smart
401161504 J. BrownJ. B

...

401161187 D. PowellD. Powell
401161187 J.J. BareaJ.J. Barea
401161187 T. Hardaway Jr.T. Hardaway Jr.
401161187 L. DoncicL. Doncic
401161187 M. KleberM. Kleber
401161187 J. JacksonJ. Jackson
401161187 D. WrightD. Wright
401161187 J. BrunsonJ. Brunson
401161187 S. CurryS. Curr
401161187 B. MarjanovicB. Marjanovic
401161187 C. LeeC. Lee
401161187 J. ReavesJ. Reaves
401161187 TE
401161187 
401161202 T. YoungT. Young
401161202 L. MarkkanenL. Markkanen
401161202 K. DunnK. Dunn
401161202 Z. LaVineZ. LaVine
401161202 T. SatoranskyT. Satoransky
401161202 D. GaffordD. Gafford
401161202 C. HutchisonC. Hutchison
401161202 L. KornetL. Kornet
401161202 R. ArcidiaconoR. Arcidiacono
401161202 S. HarrisonS. Harrison
401161202 C. WhiteC. White
401161202 D. ValentineD. Valentine
401161202 C. FelicioC. Felicio
401161202 TE
401161202 
401161202 J. HartJ. Hart
401161202 B. IngramB. Ingram
401161202 D. FavorsD. Favors
401161202 L. BallL. Ball
401161202 J. RedickJ. Redick
401161202 N. MelliN. Melli
401161202 

401161604 W. Carter Jr.W. Carter Jr.
401161604 S. HarrisonS. Harrison
401161604 C. WhiteC. White
401161604 D. ValentineD. Valentine
401161604 T. YoungT. Young
401161604 D. GaffordD. Gafford
401161604 O. Porter Jr.O. Porter Jr.
401161604 T. SatoranskyT. Satoransky
401161604 C. FelicioC. Felicio
401161604 R. ArcidiaconoR. Arcidiacono
401161604 TE
401161604 
401160627 K. LoveK. Love
401160627 C. OsmanC. Osman
401160627 T. ThompsonT. Thompson
401160627 C. SextonC. Sexton
401160627 D. GarlandD. Garland
401160627 L. Nance Jr.L. Nance Jr.
401160627 M. DellavedovaM. Dellavedova
401160627 J. ClarksonJ. Clarkson
401160627 K. Porter Jr.K. Porter Jr.
401160627 D. WadeD. Wade
401160627 T. CookT. Cook
401160627 B. KnightB. Knight
401160627 TE
401160627 
401160627 A. GordonA. Gordon
401160627 J. IsaacJ. Isaac
401160627 N. VucevicN. Vucevic
401160627 D.J. AugustinD.J. Augustin
401160627 E. FournierE. Fournier
401160627 A. AminuA. Aminu
401160627 M. BambaM. Bamba
401160627 M. Carter-WilliamsM. Carter-W

401161062 J. CrowderJ. Crowder
401161062 J. ValanciunasJ. Valanciunas
401161062 J. MorantJ. Morant
401161062 D. BrooksD. Brooks
401161062 S. HillS. Hill
401161062 K. AndersonK. Anderson
401161062 B. ClarkeB. Clarke
401161062 T. JonesT. Jones
401161062 G. AllenG. Allen
401161062 D. MeltonD. Melton
401161062 B. CabocloB. Caboclo
401161062 M. GuduricM. Guduric
401161062 TE
401161062 
401161062 K. LoveK. Love
401161062 C. OsmanC. Osman
401161062 T. ThompsonT. Thompson
401161062 C. SextonC. Sexton
401161062 D. GarlandD. Garland
401161062 J. HensonJ. Henson
401161062 L. Nance Jr.L. Nance Jr.
401161062 M. DellavedovaM. Dellavedova
401161062 J. ClarksonJ. Clarkson
401161062 K. Porter Jr.K. Porter Jr.
401161062 A. McKinnieA. McKinnie
401161062 A. ZizicA. Zizic
401161062 B. KnightB. Knight
401161062 TE
401161062 
401161086 J. CollinsJ. Collins
401161086 D. HunterD. Hunter
401161086 B. FernandoB. Fernando
401161086 T. YoungT. Young
401161086 K. HuerterK. Huerter
401161086 J. ParkerJ. Parker
40116

401161480 K. OlynykK. Olynyk
401161480 B. AdebayoB. Adebayo
401161480 D. Jones Jr.D. Jones Jr.
401161480 K. NunnK. Nunn
401161480 D. RobinsonD. Robinson
401161480 U. HaslemU. Haslem
401161480 C. SilvaC. Silva
401161480 J. CrowderJ. Crowder
401161480 S. HillS. Hill
401161480 G. DragicG. Dragic
401161480 A. IguodalaA. Iguodala
401161480 J. ButlerJ. Butler
401161480 T. HerroT. Herro
401161480 TE
401161480 
401161491 K. OlynykK. Olynyk
401161491 B. AdebayoB. Adebayo
401161491 D. Jones Jr.D. Jones Jr.
401161491 K. NunnK. Nunn
401161491 D. RobinsonD. Robinson
401161491 C. SilvaC. Silva
401161491 J. CrowderJ. Crowder
401161491 G. DragicG. Dragic
401161491 A. IguodalaA. Iguodala
401161491 U. HaslemU. Haslem
401161491 J. ButlerJ. Butler
401161491 S. HillS. Hill
401161491 G. VincentG. Vincent
401161491 TE
401161491 
401161491 K. LoveK. Love
401161491 C. OsmanC. Osman
401161491 A. DrummondA. Drummond
401161491 C. SextonC. Sexton
401161491 D. GarlandD. Garland
401161491 L. Nance Jr.L. Nance Jr.
40

AttributeError: 'Series' object has no attribute 'encode'

In [2]:
def to_int(x):
    if x == '' or x == '--' or pd.isnull(x):
        return 0
    else:
        return int(x)

In [3]:
np.unique(players1['OREB'])

array(['', '--', '0', '1', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '2', '20', '21', '22', '23', '3', '4', '5', '6', '7',
       '8', '9'], dtype=object)

In [4]:
for i in players1.columns:
    try:
        players1[i] = players1[i].apply(to_int)
    except:
        pass

/Users/Kevin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
players1.dtypes

team        object
player      object
MIN          int64
FG          object
3PT         object
FT          object
OREB         int64
DREB         int64
REB          int64
AST          int64
STL          int64
BLK          int64
TO           int64
PF           int64
+/-          int64
PTS          int64
position    object
dtype: object

In [7]:
players1[players1.player == "L. JamesL. James"].PTS.mean()

27.451219512195124

In [72]:
games1[].head()

,date,home_team,visit_team,home_team_score,visit_team_score
game_ids,,,,,
401160630,2019-10-23,phi,bos,107,93
401160659,2019-10-25,bos,tor,112,106
401160671,2019-10-26,ny,bos,95,118
401160700,2019-10-30,bos,mil,116,105
401160714,2019-11-01,bos,ny,104,102


In [77]:
year=2019

#Removed path
games1= pickle.load(open('full_df.p','rb'))
games1.set_index('game_ids',inplace=True)

BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'
print(games1.index[0])
request = requests.get(BASE_URL.format(games1.index[0]))

table = bs(request.text,'lxml').find('table', class_='mod-data')
heads = table.find_all("thead")

headers = heads[0].find_all('tr')[0].find_all('th')[1:]
headers = [th.text for th in headers]
columns = ['id', 'team', 'player'] + headers


players = pd.DataFrame(columns=columns)

def get_players(players, team_name):
    array = np.zeros((len(players), len(headers)+1), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        array[i, 0] = cols[0].text.split(',')[0]
        for j in range(1, len(headers) + 1):
            if not cols[1].text.startswith('DNP'):
                array[i, j] = cols[j].text

    frame = pd.DataFrame(columns=columns)
    for x in array:
        line = np.concatenate(([index, team_name], x)).reshape(1,len(columns))
        new = pd.DataFrame(line, columns=frame.columns)
        frame = frame.append(new)
    return frame

for index, row in games1.loc['401161022'].iterrows():
    try:
        print(index)
        request = requests.get(BASE_URL.format(index))
        
        table = bs(request.text,'lxml').find_all('table', class_='mod-data')
        heads1 = table[0].find_all('thead')
        bodies1 = table[0].find_all('tbody')
        heads2 = table[1].find_all('thead')
        bodies2 = table[1].find_all('tbody')


        #team_1 = heads[0].th.text #trouble with getting team names
        team_1 = games1['visit_team'][index]

        team_1_players = bodies1[0].find_all('tr') + bodies1[1].find_all('tr')

        team_1_players = get_players(team_1_players, team_1)
        players = players.append(team_1_players)


        #team_2 = heads[3].th.text #trouble with getting team names
        team_2 = games1['home_team'][index]
        #team_2_players = bodies[3].find_all('tr') + bodies[4].find_all('tr')
        team_2_players = bodies2[0].find_all('tr') + bodies2[1].find_all('tr')
        team_2_players = get_players(team_2_players, team_2)
        players = players.append(team_2_players)
    except Exception as e:
        print(e)
        pass

players = players.set_index('id')
players['position'] = players.player.apply(lambda x: x[-2:]  if x[-1] != 'C' else x[-1])
players['player'] = players.player.apply(lambda x: x[:-2] if x[-1] != 'C' else x[:-1])

#Trying something else

def conversion(x):
    if type(x)!=float:
        return unidecode(x)
    else:
        return x

players=players.applymap(conversion)

#Remove some useless nulls
players1 = players[pd.notnull(players.MIN)]

players1.to_csv('/Users/Kevin/Desktop/fanduel/boxscores'+str(year)+'.csv')



401160630
401161022
401161022


AttributeError: ("'Series' object has no attribute 'encode'", 'occurred at index team')

In [76]:
year=2019

#Removed path
games1= pickle.load(open('full_df.p','rb'))
games1.set_index('game_ids',inplace=True)

BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'
print(games1.index[0])
request = requests.get(BASE_URL.format(games1.index[0]))

table = bs(request.text,'lxml').find('table', class_='mod-data')
heads = table.find_all("thead")

headers = heads[0].find_all('tr')[0].find_all('th')[1:]
headers = [th.text for th in headers]
columns = ['id', 'team', 'player'] + headers


players = pd.DataFrame(columns=columns)

def get_players(players, team_name):
    array = np.zeros((len(players), len(headers)+1), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        array[i, 0] = cols[0].text.split(',')[0]
        for j in range(1, len(headers) + 1):
            if not cols[1].text.startswith('DNP'):
                array[i, j] = cols[j].text

    frame = pd.DataFrame(columns=columns)
    for x in array:
        line = np.concatenate(([index, team_name], x)).reshape(1,len(columns))
        new = pd.DataFrame(line, columns=frame.columns)
        frame = frame.append(new)
    return frame

for index, row in games1.loc['401160990'].iterrows():
    try:
        print(index)
        request = requests.get(BASE_URL.format(index))
        
        table = bs(request.text,'lxml').find_all('table', class_='mod-data')
        heads1 = table[0].find_all('thead')
        bodies1 = table[0].find_all('tbody')
        heads2 = table[1].find_all('thead')
        bodies2 = table[1].find_all('tbody')


        #team_1 = heads[0].th.text #trouble with getting team names
        team_1 = games1['visit_team'][index]

        team_1_players = bodies1[0].find_all('tr') + bodies1[1].find_all('tr')

        team_1_players = get_players(team_1_players, team_1)
        players = players.append(team_1_players)


        #team_2 = heads[3].th.text #trouble with getting team names
        team_2 = games1['home_team'][index]
        #team_2_players = bodies[3].find_all('tr') + bodies[4].find_all('tr')
        team_2_players = bodies2[0].find_all('tr') + bodies2[1].find_all('tr')
        team_2_players = get_players(team_2_players, team_2)
        players = players.append(team_2_players)
    except Exception as e:
        print(e)
        pass

players = players.set_index('id')
players['position'] = players.player.apply(lambda x: x[-2:]  if x[-1] != 'C' else x[-1])
players['player'] = players.player.apply(lambda x: x[:-2] if x[-1] != 'C' else x[:-1])

#Trying something else

def conversion(x):
    if type(x)!=float:
        return unidecode(x)
    else:
        return x

players=players.applymap(conversion)

#Remove some useless nulls
players1 = players[pd.notnull(players.MIN)]

players1.to_csv('/Users/Kevin/Desktop/fanduel/boxscores'+str(year)+'.csv')



401160630


AttributeError: 'Series' object has no attribute 'iterrows'

In [78]:
games1.loc['401161022']

,date,home_team,visit_team,home_team_score,visit_team_score
game_ids,,,,,
401161022,2019-12-14,phx,sa,119,121
401161022,2019-12-14,sa,phx,121,119


In [79]:
games1.loc['401160990']

date                2019-12-09 00:00:00
home_team                            no
visit_team                          det
home_team_score                     103
visit_team_score                    105
Name: 401160990, dtype: object